# Model Deployment Tutorial
This notebook demonstrates how to deploy a machine learning model using FastAPI.

## Contents
1. Train a Simple Model
2. Model Serialization
3. Create FastAPI Application
4. Test API Endpoints

In [ ]:
import numpy as np
import pickle
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from fastapi import FastAPI
from pydantic import BaseModel

## 1. Train a Simple Model

In [ ]:
# Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=4, n_classes=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

# Evaluate model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f'Training accuracy: {train_score:.4f}')
print(f'Testing accuracy: {test_score:.4f}')

## 2. Model Serialization

In [ ]:
def save_model(model, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)

def load_model(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

# Save model
save_model(model, 'model.pkl')

# Load model (for demonstration)
loaded_model = load_model('model.pkl')

# Verify loaded model
print('Loaded model score:', loaded_model.score(X_test, y_test))

## 3. Create FastAPI Application

In [ ]:
# Define request/response models
class PredictionInput(BaseModel):
    features: list[float]

class PredictionOutput(BaseModel):
    prediction: int
    probability: float

# Create FastAPI app
app = FastAPI(title='ML Model API')

# Load model globally
model = load_model('model.pkl')

@app.post('/predict', response_model=PredictionOutput)
async def predict(input_data: PredictionInput):
    # Convert input to numpy array
    features = np.array(input_data.features).reshape(1, -1)
    
    # Make prediction
    prediction = model.predict(features)[0]
    probability = model.predict_proba(features).max()
    
    return PredictionOutput(
        prediction=int(prediction),
        probability=float(probability)
    )

## 4. Test API Endpoint
To test the API, you would typically run the FastAPI application using uvicorn:
```bash
uvicorn app:app --reload
```

Then you can make requests using curl or Python requests:

In [ ]:
import requests

# Example request
data = {
    'features': X_test[0].tolist()
}

# Make request (uncomment when API is running)
# response = requests.post('http://localhost:8000/predict', json=data)
# print('API Response:', response.json())